In [1]:
from absbox import API,EnginePath

localAPI = API(EnginePath.DEV, lang='english', check=False)

Connecting engine server -> https://absbox.org/api/dev

✅Connected, local lib:0.28.7, server:0.28.21

In [2]:
from absbox.examples import mixedAsset_test01

#### import a mix-asset-type deal

Lets import test case , which has two pools ,`PoolA` , `PoolB`, which holds `Morgage` and `Loan` respectively

In [3]:
mixedAsset_test01.pool

{'PoolA': {'assets': [['Mortgage',
    {'originBalance': 2200,
     'originRate': ['fix', 0.045],
     'originTerm': 30,
     'freq': 'Monthly',
     'type': 'Level',
     'originDate': '2021-02-01'},
    {'currentBalance': 2200,
     'currentRate': 0.08,
     'remainTerm': 30,
     'status': 'current'}]]},
 'PoolB': {'assets': [['Loan',
    {'originBalance': 80000,
     'originRate': ['floater',
      0.045,
      {'index': 'SOFR3M', 'spread': 0.01, 'reset': 'QuarterEnd'}],
     'originTerm': 60,
     'freq': 'Monthly',
     'type': 'i_p',
     'originDate': '2021-02-01'},
    {'currentBalance': 65000,
     'currentRate': 0.06,
     'remainTerm': 60,
     'status': 'Current'}]]}}

#### Set assumption by Pool Name

In [4]:
r = localAPI.run(mixedAsset_test01
                 ,poolAssump = ("ByName"
                                ,{"PoolA":
                                      (("Mortgage",{"CDR":0.02} ,None, None, None)
                                       ,None
                                       ,None)
                                ,"PoolB":
                                  (("Loan",{"CDR":0.01} ,None, None, None)
                                   ,None
                                   ,None)}
                                )
                 ,runAssump = [("interest",("LIBOR6M",0.04)
                                           ,("SOFR3M",0.04))
                               ,("inspect",("MonthEnd",("poolBalance","PoolB"))
                                          ,("MonthEnd",("poolBalance","PoolA"))
                                )]
                 ,read=True)

TypeError: tuple indices must be integers or slices, not str

In [8]:
from lenses import lens

In [22]:
def isMixedDeal(x: dict) -> bool:
    ''' check if pool of deals has mixed asset types'''    
    if 'assets' in x or 'cashflow' in x:
        return False
    if '清单' in x or '归集表' in x:
        return False
    if 'deals' in x:
        return False
    assetTags = x & lens.Values().Each()[1][0][0].collect()
    if len(set(assetTags)) > 1:
        return True
    return False

In [23]:
mixedAsset_test01.pool & lens.Values().Each()[1][0][0].collect()

['Mortgage', 'Loan']

In [24]:
isMixedDeal(mixedAsset_test01.pool)

True

In [5]:
r['pool']['flow']['PoolA'].head()

NameError: name 'r' is not defined

In [6]:
r['pool']['flow']['PoolB'].head()

NameError: name 'r' is not defined

### Deal with Mixed Asset Type